In [58]:
from utils import models
from utils.models import FioBase
import subprocess

In [ ]:
f = FioBase()
fio_params = [("--bs", "8k"), 
              ("--name", "job1"), 
              ("--ioengine","libaio"), 
              ("--iodepth", "1"), 
              ("--size","20m"), 
              ("--filename","/tmp/test.fio"), 
              ("--direct", "1"),
              ("--rw", "randread"),
              ("--runtime", "5"),
              "--group_reporting",
              "--time_based"]
param_list: list = ["--output-format=json"]
for param in fio_params:
    print(type(param))
    if type(param) == str:
        param_list.append(param)
    else:
        param_list.append(param[0] + "=" + param[1])
param_list

In [ ]:
fio_process = subprocess.run(["fio"] + param_list,
                                capture_output=True)
fio_process.stdout

In [60]:
import json
from utils.fio_runner import FioRunner
json_result = json.loads(fio_process.stdout)
fio = FioRunner()
# dir(fio)
fiobase = fio.stdout_to_FioBase(fio_process.stdout)
str(fiobase)

'{"read_bandwidth": 27295, "read_latency": 292.341675, "read_iops": 3411.917725, "write_bandwidth": 0, "write_latency": 0.0, "write_iops": 0.0, "total_bandwidth": 27295, "timestamp": "Mon Dec 19 15:28:44 2022", "duration": 6, "total_iops": 3411.917725, "ERROR_CODE": null}'

# FioOptimizer Class
## from optimal-io-depth-search.py

In [81]:
from typing import Any, Union
from math import floor
from utils.parsers import get_file, parse_config
from utils.models import FioBase
from utils.converters import average
from argparse import ArgumentParser, Namespace

In [100]:

class FioOptimizer:
    """
    //TODO//
    """
    best_run: FioBase
    optimal_queue_depth: int
    config: dict
    runs: dict

    def __init__(self,
                 runs: dict = None,
                 best_run: FioBase = None,
                 optimal_queue_depth: int = 0,
                 config: dict = None, 
                 min: int = 1,
                 max: int = 65536):

        self.runs = runs
        self.config = config
        self.best_run: FioBase = best_run
        self.optimal_queue_depth: int = optimal_queue_depth
        self.min: int = min
        self.max: int = max

    def find_optimal_iodepth(self) -> None:
        is_optimial: bool = False
        starting_io_depths: list = [self.min, self.max]    # gotta start some where

        for io_depth in starting_io_depths:
            self.config['io_depth'] = io_depth
            fio_run = self.prepare_and_run_fio(io_depth=io_depth)
            self.runs[io_depth] = fio_run

        while not is_optimial: 
            if max.io_depth - min.io_depth <= 1:
                self.best_run = max if max.total_iops > min.total_iops else min 
                is_optimial: bool = True
            else:
                next_iodepth: int = floor(average(max.io_depth, min.io_depth))
                self.config['io_depth'] = next_iodepth
                fio_run = self.prepare_and_run_fio(io_depth=next_iodepth)
                self.runs[next_iodepth] = fio_run
                if average(self.min.total_iops, fio_run.total_iops) > average(fio_run.total_iops, self.max.total_iops):
                    self.max = fio_run.io_depth
                else: 
                    self.min = fio_run.io_depth

        
        # test limits (min, max)
        # test midpoint between limits
        # average results of min+midpoint and max+midpoint
        # 
        # divide total from min and max into n buckets, then go test all of them,
        # sort them 
        # range(min, max, (abs(max-min))/5)
        # slices = 5 
        # 10, 20, 2 # ((20-10)/slices = 2)
        # k = [10, 12, 14, 16, 18]
        # when I do the loop, test if k[n] is already tested
        # find the best results from min, max, k[]
        

        # while not is_optimial:
        #     sorted_runs_by_iops: list[FioBase] = sorted(self.runs, key=self.runs.get('total_iops'), reverse=True)
        #     # are we going up or down?
        #     if sorted_runs_by_iops[0].io_depth - sorted_runs_by_iops[1].io_depth <= 1:
        #         is_optimial = True
        #         self.optimal_queue_depth = sorted_runs_by_iops[0]
        #         return 
        #     else: 
        #         next_io_depth = ((sorted_runs_by_iops[0].io_depth + sorted_runs_by_iops[1].io_depth) // 2)
        #         if next_io_depth in self.runs:
        #             next_io_depth = 
        #         else:
        #             self.prepare_and_run_fio(io_depth=next_io_depth)

    def prepare_and_run_fio(self, io_depth: int) -> FioBase:
        print("Running Test with IO Depth = {0}".format(io_depth))
        param_list = [f"{k}={v}" if v else f"{k}" for k, v in self.config.items()]
        print(f"args: {param_list}")
        # fio_run_process: object = FioBase.run_fio(params=param_list)
        fio_run_process = subprocess.run(['fio'] + param_list, capture_output=True)
        fio_run: FioBase = FioBase()
        fio_run.parse_stdout(fio_run_process.stdout)
        return fio_run

# pick a job count equal to number of CPUs
#   Grab NIC speed as "target" maximum
# decide which test to iterate next

    def compare_run_throughput(self, new_run: FioBase) -> bool:
        if new_run.total_bandwidth > self.best_run.total_bandwidth:
            self.best_run = new_run
            return True
        return False

f = FioOptimizer()


In [113]:
fio_params = {"--bs": "8k", 
              "--name": "job1", 
              "--ioengine":"libaio", 
              "--iodepth": "1", 
              "--size": "2m", 
              "--filename": "./test.fio", 
              "--direct": "1",
              "--rw": "randread",
              "--runtime": "5s",
              "--group_reporting": None,
              "--output-format": "json"}
param_list: list = [f"{k}={v}" for k, v in fio_params.items()]
# print(fio_params)
# print(param_list)

fb = FioBase()
fio= FioOptimizer(config=fio_params, min=6, max= 10) 


print(f"Fio Config: {fio.config}")
# fio.find_optimal_iodepth()
# a = fio.best_run.prepare_args(params=fio.config)
# fio_process = subprocess.run(['fio'] + [f"{k}={v}" if v else f"{k}" for k, v in fio.config.items()], capture_output=True)
# print(fio_process)

# param_list = [f"{k}={v}" if v else f"{k}" for k, v in fio.config.items()]
# print(param_list)
# fio_process = subprocess.run(['fio'] + param_list, capture_output=True)
# fio_process.stdout
# fio_run: FioBase = FioBase()
# fio_run.parse_stdout(raw_stdout=fio_process.stdout)
print(fio.prepare_and_run_fio(io_depth=16))

Fio Config: {'--bs': '8k', '--name': 'job1', '--ioengine': 'libaio', '--iodepth': '1', '--size': '2m', '--filename': './test.fio', '--direct': '1', '--rw': 'randread', '--runtime': '5s', '--group_reporting': None, '--output-format': 'json'}
Running Test with IO Depth = 16
args: ['--bs=8k', '--name=job1', '--ioengine=libaio', '--iodepth=1', '--size=2m', '--filename=./test.fio', '--direct=1', '--rw=randread', '--runtime=5s', '--group_reporting', '--output-format=json']
{"read_bandwidth": 20897, "read_latency": 378.992188, "read_iops": 2612.244873, "write_bandwidth": 0, "write_latency": 0.0, "write_iops": 0.0, "total_bandwidth": 20897, "timestamp": "Wed Dec 21 05:53:13 2022", "duration": 1, "total_iops": 2612.244873, "ERROR_CODE": null}


In [114]:
fio.max
fio.find_optimal_iodepth()

Running Test with IO Depth = 6
args: ['--bs=8k', '--name=job1', '--ioengine=libaio', '--iodepth=1', '--size=2m', '--filename=./test.fio', '--direct=1', '--rw=randread', '--runtime=5s', '--group_reporting', '--output-format=json', 'io_depth=6']


RuntimeError: Failed to Parse FIO Output